##### import library

In [16]:
import sys
sys.path.append("../..")
# sys.path.append("../../../data/")

In [17]:
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import numpy as np

##### Function to write DataFrame to FASTA format

In [18]:
def dataframe_to_fasta(df, column, output_file):
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            # Write the FASTA header with key
            f.write(f">{row['key']}\n")
            # Write the sequence with REF_FLANK
            f.write(f"{row[column]}\n")

### Read the ecoli metadata with kgain scores 

In [19]:
df = pd.read_excel("../../data/MetaData_ecoli_final.xlsx", sheet_name= "Gain score")
df.head()

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain
0,63,intergenic,A->C,A,C,noncoding,p6,-10.385914
1,201,thrL,T->G,T,G,missense,p6,-1.750693
2,241,thrL,A->C,A,C,missense,m1,-0.911836
3,309,thrA,T->G,T,G,noncoding,m1,4.009052
4,322,thrA,A->G,A,G,noncoding,p3,1.532477


In [20]:
df[df.duplicated(subset=['Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label'])]

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain


In [21]:
file_path_Ecoli = "../../data/REL606.fna"

###### Read fasta file

##### Read fasta as sequence

In [22]:
import kaos
sequence = kaos.read_fasta(file_path_Ecoli)

##### Generate flanking regions for reference and alternate sequences with 50 bases left and 50 bases right ( Exception can be handled with adjustable flank size)

In [23]:

def generate_flanking_regions(sequence, snp_positions, alt_bases, flanking_length_left=50, flanking_length_right=50):
    flanking_regions_ref = []
    flanking_regions_alt = []
    for i, pos in enumerate(snp_positions):
        pos = int(pos)  # Ensure the position is integer
        start = max(0, pos - flanking_length_left - 1)
        end = min(pos + flanking_length_right, len(sequence))

        # Extract the reference flanking region
        flanking_region_ref = sequence[start:end]

        # Construct the alternate region with the mutated base
        evolved_nuc = alt_bases[i]  # Alternate base
        modified_region = (
            flanking_region_ref[:flanking_length_left] +
            evolved_nuc +
            flanking_region_ref[flanking_length_left + 1:]
        )

        flanking_regions_ref.append(flanking_region_ref)
        flanking_regions_alt.append(modified_region)

    return flanking_regions_ref, flanking_regions_alt

##### Function to process the FASTA sequence and generate reference and alternate flank

In [24]:

def process_fasta_and_snp_data(data, flanking_length_left=50, flanking_length_right=50):

    data = data.copy()
    
    snp_positions = data['Position']
    ref_bases = data['Ref_allele']
    alt_bases = data['Alt_allele']
    
    # Generate flanking regions for both ref and alt sequences with specified left and right lengths
    flanking_regions_ref, flanking_regions_alt = generate_flanking_regions(sequence, snp_positions, alt_bases, flanking_length_left, flanking_length_right)
    data['REF_FLANK'] = flanking_regions_ref
    data['ALT_FLANK'] = flanking_regions_alt
    
    return data

##### Dataframe with reference and alternate flank with flank size 101

In [25]:
process_df_with_flank_50 = process_fasta_and_snp_data(data = df, flanking_length_left=50, flanking_length_right=50)

In [26]:
process_df_with_flank_50.shape

(36922, 10)

##### Identify the row where the flank length is not equal to 101

In [27]:
process_df_with_flank_50[(process_df_with_flank_50['REF_FLANK'].str.len() != 101) | (process_df_with_flank_50['ALT_FLANK'].str.len() != 101)]

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain,REF_FLANK,ALT_FLANK
36921,4629764,lasT,T->G,T,G,missense,m1,-9.587243,GGGCTTTTAGAGCAACGAGACACGGCAATGTTGCACCGTTTGCTGC...,GGGCTTTTAGAGCAACGAGACACGGCAATGTTGCACCGTTTGCTGC...


#### Note: we found one mutation where flank size is 99.

In [28]:
# process_df_with_flank_50 = process_df_with_flank_50[(process_df_with_flank_50['REF_FLANK'].str.len() == 101) & (process_df_with_flank_50['ALT_FLANK'].str.len() == 101)]

In [29]:
# process_df_with_flank_50.shape

##### Save FASTA file with Alternate flank

In [32]:

process_df_with_alt_flank_50 = process_df_with_flank_50.copy()
process_df_with_alt_flank_50['key'] = process_df_with_alt_flank_50[['Position', 'Gene', 'Allele', 'Ref_allele', 'Alt_allele', 'Annotation',
       'label']].astype(str).agg('_'.join, axis=1)
process_df_with_alt_flank_50.drop(['Position', 'Gene',
 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label','REF_FLANK','accumulated_gain'], axis=1, inplace=True)
process_df_with_alt_flank_50# Define the output FASTA file path
output_fasta_file = 'alt_output.fasta'
# Call the function to convert DataFrame to FASTA
dataframe_to_fasta(process_df_with_alt_flank_50, "ALT_FLANK", output_fasta_file)
print(f"Alternate FASTA file has been created at {output_fasta_file}")

Alternate FASTA file has been created at alt_output.fasta


##### Save FASTA file with Reference flank

In [31]:
# process_df_with_ref_flank_50 = process_df_with_flank_50.copy()
# process_df_with_ref_flank_50['key'] = process_df_with_ref_flank_50[['Position', 'Gene', 'Allele', 'Ref_allele', 'Alt_allele', 'Annotation',
#        'label']].astype(str).agg('_'.join, axis=1)
# process_df_with_ref_flank_50.drop(['Position', 'Gene',
#  'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label','ALT_FLANK','accumulated_gain'], axis=1, inplace=True)
# # process_df_with_ref_flank_50
# output_fasta_file =  'ref_output.fasta'
# # Call the function to convert DataFrame to FASTA
# dataframe_to_fasta(process_df_with_ref_flank_50, "REF_FLANK", output_fasta_file)

# print(f"Reference FASTA file has been created at {output_fasta_file}")
